training

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pickle
import random

In [7]:

pip install numpy==1.26.4


Note: you may need to restart the kernel to use updated packages.Collecting numpy==1.26.4
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2



  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

X = np.load('hand_landmarks.npy')  
y = np.load('labels.npy')          


print("Features shape:", X.shape)  
print("Labels shape:", y.shape)


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("LabelEncoder saved successfully.")

X = X / np.max(X)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

def augment_landmarks(X, noise_factor=0.01, shift_factor=0.05):
    X_aug = X.copy()

    X_aug += np.random.normal(0, noise_factor, X_aug.shape)

    shift = np.random.uniform(-shift_factor, shift_factor, size=X_aug.shape)
    X_aug += shift
    
    return X_aug

X_train_aug = augment_landmarks(X_train)

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2])),  
    layers.LSTM(128, return_sequences=False), 
    layers.BatchNormalization(),              
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),                       
    layers.Dense(len(np.unique(y_encoded)), activation='softmax')  
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

0checkpoint = ModelCheckpoint(
    'best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1
)
early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5, verbose=1
)

history = model.fit(
    X_train_aug, y_train,  
    validation_data=(X_test, y_test), 
    epochs=100, batch_size=50, 
    callbacks=[checkpoint, early_stopping, reduce_lr],
    class_weight=class_weights_dict  

model.save('hope.keras')
print("Model trained and saved successfully.")


In [5]:
X = np.load('hand_landmarks.npy')
y = np.load('labels.npy')

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

X = np.array(X)
X = X / np.max(X)  # or normalize per-sample as mentioned above

# Reshape for LSTM
X = X.reshape((-1, 1, X.shape[1]))  # (samples, timesteps=1, features=63)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Data Augmentation (optional)
X_train_aug = X_train + np.random.normal(0, 0.01, X_train.shape)

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Build model
model = models.Sequential([
    layers.Input(shape=(1, X.shape[2])),
    layers.LSTM(128),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5)

# Train
history = model.fit(
    X_train_aug, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=50,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    class_weight=class_weights_dict
)

model.save('hope_v2.keras')
print("Model trained and saved successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'hand_landmarks.npy'